# Identifying schools that have inaccurate data

Many schools in this data contain inaccurate values for disipline data due to data entry errors. This notebook documents the various data entry issues with the data. All rows printed in this notebook are removed in `clean/src/clean_crdc_data.py`

In [10]:
import pandas as pd
import constants


This input file was copied from the output of the `clean` task without calling the function `drop_data_entry_errors`

In [11]:
df = pd.read_csv("input/crdc-referrals-arrests-cleaned.csv", low_memory=False)


In [12]:
df.groupby("year").COMBOKEY.nunique()


year
2013    95507
2015    29383
2017    97632
Name: COMBOKEY, dtype: int64

In [13]:
df.columns.tolist()

['COMBOKEY',
 'LEA_STATE',
 'LEAID',
 'LEA_NAME',
 'SCHID',
 'SCH_NAME',
 'JJ',
 'SCH_STATUS_ALT',
 'SCH_ENR_HI_M',
 'SCH_ENR_HI_F',
 'SCH_ENR_AM_M',
 'SCH_ENR_AM_F',
 'SCH_ENR_AS_M',
 'SCH_ENR_AS_F',
 'SCH_ENR_HP_M',
 'SCH_ENR_HP_F',
 'SCH_ENR_BL_M',
 'SCH_ENR_BL_F',
 'SCH_ENR_WH_M',
 'SCH_ENR_WH_F',
 'SCH_ENR_TR_M',
 'SCH_ENR_TR_F',
 'TOT_ENR_M',
 'TOT_ENR_F',
 'SCH_IDEAENR_HI_M',
 'SCH_IDEAENR_HI_F',
 'SCH_IDEAENR_AM_M',
 'SCH_IDEAENR_AM_F',
 'SCH_IDEAENR_AS_M',
 'SCH_IDEAENR_AS_F',
 'SCH_IDEAENR_HP_M',
 'SCH_IDEAENR_HP_F',
 'SCH_IDEAENR_BL_M',
 'SCH_IDEAENR_BL_F',
 'SCH_IDEAENR_WH_M',
 'SCH_IDEAENR_WH_F',
 'SCH_IDEAENR_TR_M',
 'SCH_IDEAENR_TR_F',
 'TOT_IDEAENR_M',
 'TOT_IDEAENR_F',
 'SCH_DISCWODIS_REF_HI_M',
 'SCH_DISCWODIS_REF_HI_F',
 'SCH_DISCWODIS_REF_AM_M',
 'SCH_DISCWODIS_REF_AM_F',
 'SCH_DISCWODIS_REF_AS_M',
 'SCH_DISCWODIS_REF_AS_F',
 'SCH_DISCWODIS_REF_HP_M',
 'SCH_DISCWODIS_REF_HP_F',
 'SCH_DISCWODIS_REF_BL_M',
 'SCH_DISCWODIS_REF_BL_F',
 'SCH_DISCWODIS_REF_WH_M',
 'SCH_DI

## Arrest or enrollment rates over 100%
Per the CRDC documentation, arrest and referral totals are supposed to represent the number of unique students who were referred or arrested. In this data, however, some schools have arrest and referral totals greater than the size of their total enrollment. 

In [24]:
df.query("year == 2015")[["COMBOKEY","LEAID", "SCHID", "SCH_NAME"]]

,COMBOKEY,LEAID,SCHID,SCH_NAME
95507,20018000075,200180,75,East High School
95508,20018000064,200180,64,Clark Middle School
95509,20018000057,200180,57,Bartlett High School
95510,20018000729,200180,729,Nicholas J. Begich Middle School
95511,20018000120,200180,120,West High School
...,...,...,...,...
124885,268036000000.0,2680360,1830,Dickinson Iron Early Childhood Education
124886,24SOP0299996,24SOP02,99996,J. DEWEESE CARTER YOUTH FACILITY
124887,25SOP0199994,25SOP01,99994,Goss II Treatment
124888,25SOP0199995,25SOP01,99995,Goss I Treatment


In [14]:
df.query("total_arrests > total_enrollment | total_referrals > total_enrollment")


UndefinedVariableError: name 'total_arrests' is not defined

## More arrests than referrals
Per the CRDC documentation, all arrests are to be counted as referrals, but not all referrals are arrests, therefore the number of arrests should never be greater than the number of referrals. 

In [ ]:
df.query("total_arrests > total_referrals")


,COMBOKEY,LEA_STATE,LEAID,LEA_NAME,SCHID,SCH_NAME,JJ,SCH_STATUS_ALT,SCH_ENR_HI_M,SCH_ENR_HI_F,...,total_arrests_hp,total_referrals_hp,total_arrests_tr,total_referrals_tr,total_arrests_idea,total_arrests_nondis,total_referrals_idea,total_referrals_nondis,total_enrollment_idea,total_enrollment_nondis
85,320048000545,NV,3200480,WASHOE COUNTY SCHOOL DISTRICT,545,SPANISH SPRINGS HIGH SCHOOL,No,No,335.0,311.0,...,0.0,0.0,4.0,2.0,17.0,50.0,2.0,4.0,263.0,2296.0
144,550852000925,WI,5508520,MADISON METROPOLITAN SCHOOL DISTRICT,925,EAST HIGH,No,No,122.0,122.0,...,0.0,0.0,4.0,0.0,8.0,0.0,2.0,2.0,323.0,1613.0
154,170993003505,IL,1709930,CITY OF CHICAGO SD 299,3505,CHICAGO INTERNATIONAL CHARTER,No,No,1172.0,1121.0,...,0.0,0.0,0.0,0.0,0.0,8.0,0.0,5.0,1211.0,8563.0
171,180363000548,IN,1803630,FORT WAYNE COMMUNITY SCHOOLS,548,MIAMI MIDDLE SCHOOL,No,No,83.0,95.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,148.0,777.0
172,180363000550,IN,1803630,FORT WAYNE COMMUNITY SCHOOLS,550,NORTH SIDE HIGH SCHOOL,No,No,86.0,92.0,...,0.0,0.0,0.0,0.0,4.0,2.0,2.0,2.0,283.0,1667.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216694,180192000249,IN,1801920,Clarksville Community School Corp,249,Clarksville Senior High School,No,No,23.0,18.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,85.0,424.0
216697,180192000248,IN,1801920,Clarksville Community School Corp,248,Clarksville Middle School,No,No,23.0,18.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,85.0,424.0
216870,180951001555,IN,1809510,Richmond Community Schools,1555,Dennis Intermediate School,No,No,35.0,31.0,...,0.0,0.0,3.0,4.0,3.0,11.0,2.0,11.0,133.0,649.0
217026,181110001798,IN,1811100,M S D Steuben County,1798,Angola Middle School,No,No,46.0,29.0,...,0.0,0.0,0.0,0.0,0.0,12.0,0.0,10.0,113.0,660.0


## Schools with very high totals and near-identical arrest and referral rates
After reaching out to Del Valle ISD outside Austin, TX, we were told their data was incorrect and that the person who did the data entry for the distict misunderstood the definitions in the discipline section. All of their schools reported extremely high arrest and referral totals, and had arrest totals that were equal or nearly equal to their referral totals. It's clear that there are several other districts that made the same error -- all have significantly high arrest totals that are identical to their referral totals

In [ ]:
df = df.assign(
    grade_category=lambda df: df.apply(
        lambda row: "high school"
        if row.max_grade in range(10, 13)
        else "middle school"
        if row.max_grade in range(7, 10)
        else "elementary school"
        if row.max_grade in range(1, 7)
        else "other",
        axis=1,
    )
)

threshold_df = (
    df.groupby(["grade_category", "year"])
    .total_referrals_arrests.quantile(0.999)
    .to_frame("threshold")
)

threshold_df


threshold
grade_category    year           
elementary school 2013     58.730
                  2015     35.522
                  2017     29.260
high school       2013    221.608
                  2015    236.100
                  2017    193.498
middle school     2013    193.800
                  2015    105.514
                  2017    137.282
other             2013     44.737
                  2015      7.772
                  2017      6.806

In [ ]:
close_vals = list(range(0, 3))
df.merge(threshold_df, left_on=["grade_category", "year"], right_index=True).query(
    "total_referrals_arrests > threshold & abs(total_arrests - total_referrals) in @close_vals"
)


,COMBOKEY,LEA_STATE,LEAID,LEA_NAME,SCHID,SCH_NAME,JJ,SCH_STATUS_ALT,SCH_ENR_HI_M,SCH_ENR_HI_F,...,total_arrests_tr,total_referrals_tr,total_arrests_idea,total_arrests_nondis,total_referrals_idea,total_referrals_nondis,total_enrollment_idea,total_enrollment_nondis,grade_category,threshold
231,271242000610,MN,2712420,FRIDLEY PUBLIC SCHOOL DISTRICT,610,FRIDLEY MIDDLE,No,No,50.0,50.0,...,17.0,17.0,80.0,275.0,80.0,275.0,124.0,824.0,middle school,193.800
14314,290462002517,MO,2904620,BELTON 124,2517,YEOKUM MIDDLE,No,No,53.0,38.0,...,12.0,12.0,53.0,361.0,53.0,361.0,84.0,719.0,middle school,193.800
70510,291376002659,MO,2913760,HARRISONVILLE R-IX,2659,HARRISONVILLE MIDDLE,No,No,8.0,11.0,...,2.0,2.0,15.0,93.0,15.0,93.0,47.0,582.0,middle school,193.800
3614,271242000611,MN,2712420,FRIDLEY PUBLIC SCHOOL DISTRICT,611,FRIDLEY SENIOR HIGH,No,No,38.0,35.0,...,20.0,20.0,89.0,408.0,89.0,408.0,140.0,869.0,high school,221.608
74157,251050001684,MA,2510500,SAUGUS,1684,SAUGUS HIGH,No,No,32.0,50.0,...,4.0,4.0,28.0,167.0,28.0,167.0,47.0,709.0,high school,221.608
14302,290462002724,MO,2904620,BELTON 124,2724,MILL CREEK UPPER ELEM.,No,No,41.0,53.0,...,4.0,4.0,27.0,98.0,27.0,98.0,102.0,791.0,elementary school,58.730
96532,550702000000.0,WI,5507020,Janesville School District,757,Craig High,No,No,77.0,71.0,...,11.0,11.0,44.0,85.0,44.0,85.0,213.0,1712.0,high school,236.100
97271,130282000000.0,GA,1302820,Henry County,1208,Henry County High School,No,No,35.0,44.0,...,0.0,0.0,18.0,111.0,18.0,111.0,167.0,1064.0,high school,236.100
106602,270566000000.0,MN,2705660,BIRD ISLAND-OLIVIA-LAKE LILLIAN,147,BOLD SENIOR HIGH,No,No,17.0,23.0,...,7.0,7.0,16.0,126.0,16.0,126.0,64.0,370.0,high school,236.100
100732,171704000000.0,IL,1717040,Woodridge SD 68,2015,Willow Creek Elem School,No,No,47.0,47.0,...,4.0,4.0,6.0,15.0,6.0,15.0,36.0,337.0,elementary school,35.522
